In [1]:
from langchain_community.utilities import SQLDatabase
import streamlit as st
from sqlalchemy import create_engine, text
import pandas as pd



In [2]:
import torch

c:\Users\aniket singh\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\_pytree.py:174: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [6]:
torch.cuda.is_available()

True

In [4]:
torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

device(type='cuda', index=0)

In [7]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("unsloth/Qwen2.5-Coder-14B-bnb-4bit")
model = AutoModelForCausalLM.from_pretrained("unsloth/Qwen2.5-Coder-14B-bnb-4bit")

c:\Users\aniket singh\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.86s/it]


ValueError: `.to` is not supported for `4-bit` or `8-bit` bitsandbytes models. Please use the model as it is, since the model has already been set to the correct devices and casted to the correct `dtype`.

In [2]:
section = pd.read_csv("section_with_licr.csv")
student = pd.read_csv("student_data_licr.csv")

In [6]:
student['student_name'] = student['student_name'].str.lower()
student['chapter_name'] = student['chapter_name'].str.lower()
student['section_name'] = student['section_name'].str.lower()
student['subject'] = student['subject'].str.lower()
student['topic_name'] = student['topic_name'].str.lower()


In [7]:
student.head()

,student_id,student_name,subject,is_active,section_id,student_level_LG_count,student_level_total_LU_count,student_level_engaged_LU_count,chapter_id,chapter_name,...,topic_video_seen,topic_reward_claim,student_LI,student_CR,chapter_LI,chapter_CR,topic_LI,topic_CR,teacher_id,section_name
0,8798,vanshika jethi,science,True,2207,71,118,82,2534,crop production and management,...,False,False,0.4,0.69,0.0,1.0,0.0,1.0,8933,class 8a
1,8798,vanshika jethi,science,True,2207,71,118,82,2534,crop production and management,...,False,False,0.4,0.69,0.0,1.0,0.0,1.0,8933,class 8a
2,8798,vanshika jethi,science,True,2207,71,118,82,2534,crop production and management,...,False,False,0.4,0.69,0.0,1.0,0.0,1.0,8933,class 8a
3,8798,vanshika jethi,science,True,2207,71,118,82,2534,crop production and management,...,False,False,0.4,0.69,0.0,1.0,0.0,1.0,8933,class 8a
4,8798,vanshika jethi,science,True,2207,71,118,82,2534,crop production and management,...,False,False,0.4,0.69,0.0,1.0,0.0,1.0,8933,class 8a


In [8]:
student.isnull().sum()

student_id                            0
student_name                          0
subject                               0
is_active                             0
section_id                            0
student_level_LG_count                0
student_level_total_LU_count          0
student_level_engaged_LU_count        0
chapter_id                            0
chapter_name                          0
ongoing_flag                          0
chapter_start_timestamp               0
chapter_end_timestamp                 0
chapter_is_active                     0
number_of_LU                          0
chapter_level_LG_count                0
chapter_level_engaged_LU_count        0
chapter_level_total_LU_count          0
topic_id                              0
topic_name                            0
topic_is_active                       0
topic_level_LG_count                  0
topic_level_engaged_LU_count          0
topic_level_total_LU_count            0
topic_enabled_timestamp               0


In [9]:
student.dropna(subset=["chapter_LI" ,"chapter_CR" ], inplace=True)

In [10]:
student.shape

(21011, 35)

In [12]:
section['section_name'] = section['section_name'].str.lower()
section['chapter_name'] = section['chapter_name'].str.lower()
section['subject'] = section['subject'].str.lower()
section['topic_name'] = section['topic_name'].str.lower()
section['school_name'] = section['school_name'].str.lower()
section['teacher_name'] = section['teacher_name'].str.lower()



In [13]:
section.head()

,section_id,section_name,grade,subject,is_active,number_of_students,section_level_LG_count,section_level_engaged_LU_count,section_level_total_LU_count,section_delta_li,...,teacher_name,section_count,section_LI,section_CR,CRQ_LI,CRQ_CR,chapter_LI,chapter_CR,topic_LI,topic_CR
0,2208,class 8b,8,science,True,36,962,1570,3060,0.614481,...,sneha kumari,1.0,68.56,51.31,52.78,83.33,48.46,73.15,52.78,77.78
1,2208,class 8b,8,science,True,36,962,1570,3060,0.614481,...,sneha kumari,1.0,68.56,51.31,52.78,83.33,48.46,73.15,63.89,61.11
2,2208,class 8b,8,science,True,36,962,1570,3060,0.614481,...,sneha kumari,1.0,68.56,51.31,52.78,83.33,48.46,73.15,50.00,61.11
3,2208,class 8b,8,science,True,36,962,1570,3060,0.614481,...,sneha kumari,1.0,68.56,51.31,52.78,83.33,48.46,73.15,72.22,52.78
4,2208,class 8b,8,science,True,36,962,1570,3060,0.614481,...,sneha kumari,1.0,68.56,51.31,52.78,83.33,48.46,73.15,26.85,88.89


In [14]:
section.dropna(subset=["chapter_LI" ,"chapter_CR" ], inplace=True)

In [15]:
section = section.applymap(lambda x: x.lower() if isinstance(x, str) else x)

C:\Users\aniket singh\AppData\Local\Temp\ipykernel_28400\4206850666.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  section = section.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [16]:
student = student.applymap(lambda x: x.lower() if isinstance(x, str) else x)

C:\Users\aniket singh\AppData\Local\Temp\ipykernel_28400\3240916915.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  student = student.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [18]:
student.head()

,student_id,student_name,subject,is_active,section_id,student_level_LG_count,student_level_total_LU_count,student_level_engaged_LU_count,chapter_id,chapter_name,...,topic_video_seen,topic_reward_claim,student_LI,student_CR,chapter_LI,chapter_CR,topic_LI,topic_CR,teacher_id,section_name
0,8798,vanshika jethi,science,True,2207,71,118,82,2534,crop production and management,...,False,False,0.4,0.69,0.0,1.0,0.0,1.0,8933,class 8a
1,8798,vanshika jethi,science,True,2207,71,118,82,2534,crop production and management,...,False,False,0.4,0.69,0.0,1.0,0.0,1.0,8933,class 8a
2,8798,vanshika jethi,science,True,2207,71,118,82,2534,crop production and management,...,False,False,0.4,0.69,0.0,1.0,0.0,1.0,8933,class 8a
3,8798,vanshika jethi,science,True,2207,71,118,82,2534,crop production and management,...,False,False,0.4,0.69,0.0,1.0,0.0,1.0,8933,class 8a
4,8798,vanshika jethi,science,True,2207,71,118,82,2534,crop production and management,...,False,False,0.4,0.69,0.0,1.0,0.0,1.0,8933,class 8a


In [19]:
section.head()

,section_id,section_name,grade,subject,is_active,number_of_students,section_level_LG_count,section_level_engaged_LU_count,section_level_total_LU_count,section_delta_li,...,teacher_name,section_count,section_LI,section_CR,CRQ_LI,CRQ_CR,chapter_LI,chapter_CR,topic_LI,topic_CR
0,2208,class 8b,8,science,True,36,962,1570,3060,0.614481,...,sneha kumari,1.0,68.56,51.31,52.78,83.33,48.46,73.15,52.78,77.78
1,2208,class 8b,8,science,True,36,962,1570,3060,0.614481,...,sneha kumari,1.0,68.56,51.31,52.78,83.33,48.46,73.15,63.89,61.11
2,2208,class 8b,8,science,True,36,962,1570,3060,0.614481,...,sneha kumari,1.0,68.56,51.31,52.78,83.33,48.46,73.15,50.00,61.11
3,2208,class 8b,8,science,True,36,962,1570,3060,0.614481,...,sneha kumari,1.0,68.56,51.31,52.78,83.33,48.46,73.15,72.22,52.78
4,2208,class 8b,8,science,True,36,962,1570,3060,0.614481,...,sneha kumari,1.0,68.56,51.31,52.78,83.33,48.46,73.15,26.85,88.89


In [14]:
student['section_name'] = student['section_name'].str.lower()

In [20]:
student.head()

,student_id,student_name,subject,is_active,section_id,student_level_LG_count,student_level_total_LU_count,student_level_engaged_LU_count,chapter_id,chapter_name,...,topic_video_seen,topic_reward_claim,student_LI,student_CR,chapter_LI,chapter_CR,topic_LI,topic_CR,teacher_id,section_name
0,8798,vanshika jethi,science,True,2207,71,118,82,2534,crop production and management,...,False,False,0.4,0.69,0.0,1.0,0.0,1.0,8933,class 8a
1,8798,vanshika jethi,science,True,2207,71,118,82,2534,crop production and management,...,False,False,0.4,0.69,0.0,1.0,0.0,1.0,8933,class 8a
2,8798,vanshika jethi,science,True,2207,71,118,82,2534,crop production and management,...,False,False,0.4,0.69,0.0,1.0,0.0,1.0,8933,class 8a
3,8798,vanshika jethi,science,True,2207,71,118,82,2534,crop production and management,...,False,False,0.4,0.69,0.0,1.0,0.0,1.0,8933,class 8a
4,8798,vanshika jethi,science,True,2207,71,118,82,2534,crop production and management,...,False,False,0.4,0.69,0.0,1.0,0.0,1.0,8933,class 8a


In [12]:
teacher = pd.read_csv("teacher_data.csv")

In [13]:
teacher.head()

,school_name,school_board,teacher_id,teacher_name,subject,grade,is_active,teacher_level_LG_count,teacher_level_engaged_LU_count,teacher_level_total_LU_count,teacher_delta_li,subject_grades_level_LG_count,subject_grades_level_engaged_LU_count,subject_grades_level_total_LU_count,subject_delta_li,section_count,sections
0,DPS Lucknow,CBSE,8924,Ankit Jaiswal,NaN,NaN,True,-1,-1,-1,0.646425,NaN,NaN,NaN,NaN,NaN,NaN
1,DPS Lucknow,CBSE,8925,Aprajita Srivastav,Maths,6.0,True,1707,2689,9687,0.630126,731.0,1333.0,4046.0,0.544896,1.0,2187
2,DPS Lucknow,CBSE,8925,Aprajita Srivastav,Maths,7.0,True,1707,2689,9687,0.630126,976.0,1356.0,5641.0,0.712766,1.0,2197
3,DPS Lucknow,CBSE,8926,Anita Singh,Maths,6.0,True,1094,1806,5560,0.605351,1094.0,1806.0,5560.0,0.605351,1.0,2189
4,DPS Lucknow,CBSE,8927,Aneeta Sharma,Maths,8.0,True,2928,4718,9571,0.618590,1657.0,2859.0,5076.0,0.576691,1.0,2203


In [21]:


db_path = "acadally.db"
engine = create_engine(f'sqlite:///{db_path}')

section.to_sql('section', engine, index=False , if_exists="replace")
student.to_sql('student', engine, index=False, if_exists="replace")
# teacher.to_sql('teacher', engine, index=False, if_exists="replace")



21011

In [22]:
db = SQLDatabase.from_uri("sqlite:///acadally.db")

In [23]:
db.get_table_info().split("\n")

['',
 'CREATE TABLE section (',
 '\tsection_id BIGINT, ',
 '\tsection_name TEXT, ',
 '\tgrade BIGINT, ',
 '\tsubject TEXT, ',
 '\tis_active BOOLEAN, ',
 '\tnumber_of_students BIGINT, ',
 '\t"section_level_LG_count" BIGINT, ',
 '\t"section_level_engaged_LU_count" BIGINT, ',
 '\t"section_level_total_LU_count" BIGINT, ',
 '\tsection_delta_li FLOAT, ',
 '\tchapter_id BIGINT, ',
 '\tchapter_name TEXT, ',
 '\tongoing_flag BOOLEAN, ',
 '\tchapter_start_timestamp TEXT, ',
 '\tchapter_end_timestamp TEXT, ',
 '\t"number_of_LU" BIGINT, ',
 '\t"chapter_level_LG_count" BIGINT, ',
 '\t"chapter_level_engaged_LU_count" BIGINT, ',
 '\t"chapter_level_total_LU_count" BIGINT, ',
 '\t"CLT_attempts_count" BIGINT, ',
 '\tchapter_delta_li FLOAT, ',
 '\t"CRQ_level_LG_count" FLOAT, ',
 '\t"CRQ_level_engaged_LU_count" FLOAT, ',
 '\t"CRQ_level_total_LU_count" FLOAT, ',
 '\t"CRQ_delta_li" FLOAT, ',
 '\ttopic_id FLOAT, ',
 '\ttopic_name TEXT, ',
 '\ttopic_is_active BOOLEAN, ',
 '\t"topic_level_LG_count" FLOAT, ',
 

In [24]:
db.run("""select count(distinct student_id) from student where section_name = 'class 8a' and teacher_id = 8933;""")

'[(36,)]'

In [ ]:
prefix="""You are an SQL expert generating queries for the given Question. Your task to generate SQL queries for the given Question using the schema and previous conversation 

        Always USE student_name from student table not from section table in the SQL query.
        Do NOT USE SUM in SQL QUERY. 

        Previous conversation:
        {chat_history}



        Only Tables Available and Do not use tables other then this:
        - section: Class level data
        - student: Individual student performance data (like student_name , student_CR )     
            
        Do not use SUM in sql query.
        CLass is same as Section



        Definitions:
        - Chapter Readiness Quiz (CRQ):
            A Chapter Readiness Quiz is a pre-reading assessment designed to evaluate a student's preparedness before starting a new chapter.
            It assesses basic knowledge, foundational concepts, and prerequisites to ensure students are ready for the upcoming material.
            These quizzes typically include questions on essential concepts, vocabulary, and prior knowledge related to the chapter content.
            CRQs enable students to self-assess their readiness and help instructors identify areas where additional guidance may be needed.


        Key Performance Metrics:
        1. Learning Index (LI) : The Learning Index measures how much a student has learned relative to the total number of completed learning units.
            It represents the percentage of completed tasks where the student demonstrated proficiency without any learning gaps.
            Think of it as a score that reflects the quality and depth of learning achieved from all completed efforts, highlighting true mastery of the content.
            Primary performance metric:
            - **`student_LI`**: Overall learning index for all chapters by an individual student.  
            - **`section_LI`**: Overall learning index for all students in a section.  
            - **`chapter_LI`**: Learning index specific to a single chapter.  
            - **`topic_LI`**: Learning index specific to a single topic.
            * Use student_LI or section_LI for overall analysis . student_LI when ask for student and section_LI when asked for section
            * Use chapter_LI/topic_LI only when specifically requested for Chapter or Topic

        2. Completion Rate (CR) : The Completion Rate is the ratio of the total number of unique learning units a student attempted (engaged) to the total number of unique learning units assigned.
            It indicates how much of the given tasks a student tried, regardless of correctness—similar to checking how much of your homework you attempted.
            Completion metric:
            - **`student_CR`**: Overall Completion percentage of all chapters and subject by a student.
            - **`section_CR`**: Overall Completion percentage of all chapters and subject for all student in a section.
            - **`chapter_CR`**: Overall Completion percentage for all topics of a chapter  
            - **`topic_CR`**: Overall Completion percentage of a single topic for a chapters

        3. Timeline Fields:
        - chapter_start_timestamp: Time at which chapter has started.
        - chapter_end_timestamp: Time at which chapter has ended. -1 if chapter not ended.
        - ongoing_flag: TRUE = Chapter is ongoing for a student / section, FALSE = Chapter is Completed for a Section / Student

        4. LU Attempts Count Fields:
            Do not use SUM they are already total counts
            Student level(student table) and Use this for student data :
            - student_level_LG_count: Sum/Total count of attempted Learning units with Learning Gap count by a student (-1=Not attempted or engaged ,others = Total Count of learning units with learning gap by student) 
            - student_level_engaged_LU_count: Sum/Total count of Learning units attempted/engaged by a  student  (-1=Not attempted or engaged, others=  Total Count of learning units attempted/engaged by a student)
            - student_level_total_LU_count : Sum/Total count of Learning units assigned to a student(-1 = NOt assigned , others= Total Count of Assigned learning units to a student)

            Section level(section table) and Use this for section/class data:
            - section_level_LG_count: Sum/Total count of attempted Learning units with Learning Gap  by a section (-1=Not attempted or engaged ,others = Total Count of learning units with learning gap by section) 
            - section_level_engaged_LU_count:Sum/Total count of Learning units attempted/engaged by a  section  (-1=Not attempted or engaged, others=  Total Count of learning units attempted/engaged by a section)
            - section_level_total_LU_count :Sum/Total count of Learning units assigned to a section (-1 = NOt assigned , others= Total Count of Assigned learning units to a student)
            
            Chapter level (count fields in student table is for student and in section table is for section)and Use this for chapter data:
            - chapter_level_LG_count: Sum/Total count of attempted Learning units with Learning Gap in specifc chapter by a section / student (-1=Not attempted or engaged ,others = Total Count of learning units in a chapter with learning gap by section / student) 
            - chapter_level_engaged_LU_count:Sum/Total count of Learning units attempted/engaged in the specifc chapter by a section / student  (-1=Not attempted or engaged, others=  Total Count of learning units attempted/engaged in a chapter by a section / student)
            - chapter_level_total_LU_count :Sum/Total count of Learning units assigned of specifc chapter to a section / student (-1 = NOt assigned , others= Total Count of Assigned learning units of a chapter to a student / section )
            
            CRQ level :
            - CRQ_level_LG_count: Sum/Total count of count of attempted Learning units with Learning Gap in a CRQ quiz by a section (-1=Not attempted or engaged,others = Total Count of learning units with learning gap in a CRQ quiz by a section) 
            - CRQ_level_engaged_LU_count:Sum/Total count of count of Learning units attempted/engaged in the CRQ quiz by a section  (-1=Not attempted or engaged, others =  Total Count of learning units attempted/engaged in a CRQ quiz by a section)
            - CRQ_level_total_LU_count :Sum/Total count of count of Assigned learning units in a CRQ quiz to a section  (-1 = NOt assigned , others = Total Count of Assigned learning units in a CRQ quiz to a section )
           
        SQL Database schema and Only use these field:
            {table_info}

        CRITICAL Query Rules:
        - Performance if for LI and progress is for CR. 
        - For assigned count always use  
        - Do not create new field which are not exists in Database schema
        - Always ensure subqueries are enclosed within parentheses `()` and are followed by an alias (e.g., `s` or `subquery_alias`).
        - Do not give parentheses like( "))" ) in the sql query.
        - Do not use chapter cr or li for topics cr and li.
        - For ongoing flag use True and False only not in text format.
        - Do not use SUM in sql query 
        - For checking for ongoing use ongoing_flag.Do not use ongoing_status.
        - Use level_engaged_LU_count for question for assigned
        - Always use student_name from student table not from section table
        - DO NOT use fields/columns not present in the schema/tables.
        - Do not use JOIN untill it is necessary
        - ALWAYS include 'teacher_id = {teacher_id}' in WHERE clause
        - Always use student table for student data 
        - Always use section table for section data  
        - DO not give where condition like this "s.section_name = 'your_section_name'" use context from previous conversation
        - Use DISTINCT to avoid duplicates
        - Check chat history for context
        - Use chapter_end_timestamp for chapter finished/Ended
        - Recent in the question means last 30 days of data
        

        Chat History rules:
        1. Look for mentions of in revious conversation:
        - Specific classes or sections name
        - Chapter names
        - Student names
        - Performance thresholds
        - Time periods
        2. Use these in the WHERE clause of follow-up queries
        3. Maintain context across related questions
        4. Words like (he/his/her/she) refer to the student_name from last conversation.
        {top_k}

        Generate only the SQL query without any explanation.
"""